## Try this Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/mlfoundry-examples/blob/main/examples/sklearn/call_center_timeseries.ipynb)

## Install dependencies

In [ ]:
! pip install --quiet "numpy>=1.0.0,<2.0.0" "pandas>=1.0.0,<2.0.0" "openpyxl>=3.0.9,<3.1.0" "matplotlib>=3.5.2,<3.6.0" "seaborn>=0.11.2,<0.12.0" "matplotlib>=3.5.2,<3.6.0" scikit-learn shap==0.40.0
! pip install -U mlfoundry

## Initialize MLFoundry Client

In [ ]:
import mlfoundry as mlf
mlf.login()

client = mlf.get_client()

# Timeseries model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
import pylab
import scipy

from sklearn.metrics import mean_absolute_error

In [ ]:
# importing the data
raw_csv_data = pd.read_excel("CallCenterData.xlsx", engine='openpyxl')

In [ ]:
# check point of data
df_comp = raw_csv_data.copy()

In [ ]:
df_comp.head()

In [ ]:
df_comp.describe()

In [ ]:
df_comp.isna().sum()

## Converting Date to numeric

In [ ]:
df_comp["timestamp"] = df_comp["month"].apply(lambda x : x.timestamp())

## Setting date as Index

In [ ]:
# taken as a date time field
df_comp.month.describe()

In [ ]:
df_comp.set_index("month", inplace=True)

In [ ]:
df_comp.head()

In [ ]:
# seeting the frequency as monthly
df_comp.asfreq('M')

In [ ]:
# seeting the frequency as monthly
df_comp = df_comp.asfreq('M')

In [ ]:
# checking for the null values
df_comp.isna().sum()

## Time Series Visualization

In [ ]:
df_comp.Healthcare.plot(figsize=(20,5), title="Healthcare")
plt.show()

In [ ]:
df_comp.Telecom.plot(figsize=(20,5), title="Telecom")
plt.show()

In [ ]:
df_comp.Banking.plot(figsize=(20,5), title="Banking")
plt.show()

In [ ]:
df_comp.Technology.plot(figsize=(20,5), title="Technology")
plt.show()

In [ ]:
df_comp.Insurance.plot(figsize=(20,5), title="Insurance")
plt.show()

## Check for normality

In [ ]:
# Density Plots
df_comp["Healthcare"].plot(kind='kde', figsize=(20, 10))
pyplot.show()

In [ ]:
# The QQ plot
scipy.stats.probplot(df_comp["Healthcare"], plot=pylab)
plt.title("QQ plot for Healthcare")
pylab.show()

## Gaussian Process

In [ ]:
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared, ConstantKernel
from sklearn.gaussian_process.kernels import RationalQuadratic

k0 = WhiteKernel(noise_level=0.3 ** 2, noise_level_bounds=(0.1 ** 2, 0.5 ** 2))

k1 = ConstantKernel(constant_value=2) * \
  ExpSineSquared(length_scale=1.0, periodicity=40, periodicity_bounds=(35, 45))

k2 = ConstantKernel(constant_value=100, constant_value_bounds=(1, 500)) * \
  RationalQuadratic(length_scale=500, length_scale_bounds=(1, 1e4), alpha= 50.0, alpha_bounds=(1, 1e3))

k3 = ConstantKernel(constant_value=1) * \
  ExpSineSquared(length_scale=1.0, periodicity=12, periodicity_bounds=(10, 15))

kernel_4  = k0 + k1 + k2 + k3

In [ ]:
mlf_run = mlf_api.create_run(project_name='timeseries-project', run_name='gp2-model')

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

gp2 = GaussianProcessRegressor(
    kernel=kernel_4, 
    n_restarts_optimizer=10, 
    normalize_y=True,
    alpha=0.0
)

In [ ]:
data_df = df_comp[["timestamp", "Healthcare"]]

In [ ]:
# train set split
test_size = 22
X = df_comp["timestamp"]
y = df_comp["Healthcare"]

x_train_t = X[:-test_size]
y_train_t = y[:-test_size]

x_test_t = X[-test_size:]
y_test_t = y[-test_size:]

df_train = df_comp[:-test_size]
df_test = df_comp[-test_size:]

plt.figure(figsize=(20,5))
plt.title('train and test sets', size=20)
plt.plot(y_train_t, label='Training set')
plt.plot(y_test_t, label='Test set', color='orange')

plt.legend()

In [ ]:
# logging dataset
mlf_run.log_dataset(df_train, data_slice=mlf.DataSlice.TRAIN)  
mlf_run.log_dataset(df_test, data_slice=mlf.DataSlice.TEST)

In [ ]:
x_train = x_train_t.values.reshape(-1, 1)
y_train = y_train_t.values.reshape(-1, 1)

x_test = x_test_t.values.reshape(-1, 1)
y_test = y_test_t.values.reshape(-1, 1)

In [ ]:
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')

In [ ]:
gp2_prior_samples = gp2.sample_y(X=x_train, n_samples=100)

fig, ax = plt.subplots()
for i in range(100):
    sns.lineplot(x=x_train[...,0], y = gp2_prior_samples[:, i], color=sns_c[1], alpha=0.2, ax=ax)
sns.lineplot(x=x_train[...,0], y=y_train[..., 0], color=sns_c[0], label='y2', ax=ax) 
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set(title='GP2 Prior Samples', xlabel='t');

In [ ]:
gp2.fit(x_train, y_train)

In [ ]:
mlf_run.log_params(gp2.get_params())    # logging params
mlf_run.log_model(gp2, mlf.ModelFramework.SKLEARN) # log model

In [ ]:
# Generate predictions.
y_pred, y_std = gp2.predict(x_train, return_std=True)


df_train['y_pred'] = y_pred
df_train['y_std'] = y_std
df_train['y_pred_lwr'] = df_train['y_pred'] - 2*df_train['y_std']
df_train['y_pred_upr'] = df_train['y_pred'] + 2*df_train['y_std']

In [ ]:
# Generate predictions.
y_pred_test, y_std_test = gp2.predict(x_test, return_std=True)


df_test['y_pred'] = y_pred_test
df_test['y_std'] = y_std_test
df_test['y_pred_lwr'] = df_test['y_pred'] - 2*df_test['y_std']
df_test['y_pred_upr'] = df_test['y_pred'] + 2*df_test['y_std']

In [ ]:
mlf_run.log_predictions(df_test[['timestamp', 'Healthcare']], list(y_pred_test))   #logging predictions

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(df_train["y_pred"])
plt.plot(df_train["Healthcare"], color='red')
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(df_test["y_pred"])
plt.plot(df_test["Healthcare"], color='red')
plt.show()

In [ ]:
print(f'R2 Score Train = {gp2.score(X=x_train, y=y_train): 0.3f}')
print(f'R2 Score Test = {gp2.score(X=x_test, y=y_test): 0.3f}')
print(f'MAE Train = {mean_absolute_error(y_true=y_train, y_pred=gp2.predict(x_train)): 0.3f}')
print(f'MAE Test = {mean_absolute_error(y_true=y_test, y_pred=gp2.predict(x_test)): 0.3f}')


In [ ]:
def mase_loss(y_test, y_pred, y_train):
    #  naive seasonal prediction
    y_train = np.asarray(y_train)
    y_pred_naive = y_train[:-1]

    # mean absolute error of naive seasonal prediction
    mae_naive = np.mean(np.abs(y_train[1:] - y_pred_naive))

    # if training data is flat, mae may be zero,
    # return np.nan to avoid divide by zero error
    # and np.inf values
    if mae_naive == 0:
        return np.nan
    else:
        return np.mean(np.abs(y_test - y_pred)) / mae_naive

masel = mase_loss(y_test, df_test["y_pred"].to_list(), df_train["Healthcare"].to_list())
mlf_run.log_metrics({'mase_loss': masel})    # logging metrics

In [ ]:
import shap
explainer = shap.KernelExplainer(gp2.predict,x_train)
shap_values = explainer.shap_values(x_test)

mlf_run.log_dataset_stats(
    df_test, 
    data_slice=mlf.DataSlice.TEST,
    data_schema=mlf.Schema(
        feature_column_names=['timestamp'],
        prediction_column_name="y_pred",
        actual_column_name="Healthcare"
    ),
    model_type=mlf.ModelType.TIMESERIES,
    shap_values=shap_values
)

In [ ]:
errors = gp2.predict(x_train) - y_train
errors = errors.flatten()
errors_mean = errors.mean()
errors_std = errors.std()

fig, ax = plt.subplots(1, 2, figsize=(12, 6)) 
sns.regplot(x=y_train.flatten(), y=gp2.predict(x_train).flatten(), ax=ax[0])
sns.distplot(a=errors, ax=ax[1])
ax[1].axvline(x=errors_mean, color=sns_c[3], linestyle='--', label=f'$\mu$')
ax[1].axvline(x=errors_mean + 2*errors_std, color=sns_c[4], linestyle='--', label=f'$\mu \pm 2\sigma$')
ax[1].axvline(x=errors_mean - 2*errors_std, color=sns_c[4], linestyle='--')
ax[1].axvline(x=errors_mean, color=sns_c[3], linestyle='--')
ax[1].legend()
ax[0].set(title='Model 2 - Train vs Predictions (Train Set)', xlabel='y_train', ylabel='y_pred');
ax[1].set(title='Model 2  - Errors', xlabel='error', ylabel=None);

In [ ]:
errors = gp2.predict(x_test) - y_test
errors = errors.flatten()
errors_mean = errors.mean()
errors_std = errors.std()

fig, ax = plt.subplots(1, 2, figsize=(12, 6)) 
sns.regplot(x=y_test.flatten(), y=gp2.predict(x_test).flatten(), ax=ax[0])
sns.distplot(a=errors, ax=ax[1])
ax[1].axvline(x=errors_mean, color=sns_c[3], linestyle='--', label=f'$\mu$')
ax[1].axvline(x=errors_mean + 2*errors_std, color=sns_c[4], linestyle='--', label=f'$\mu \pm 2\sigma$')
ax[1].axvline(x=errors_mean - 2*errors_std, color=sns_c[4], linestyle='--')
ax[1].axvline(x=errors_mean, color=sns_c[3], linestyle='--')
ax[1].legend()
ax[0].set(title='Model 2 - Test vs Predictions (Test Set)', xlabel='y_test', ylabel='y_pred');
ax[1].set(title='Model 2  - Errors', xlabel='error', ylabel=None);

## Difference (1)

In [ ]:
mlf_run2 = mlf_api.create_run(project_name='timeseries-project', run_name='gp1-model')

In [ ]:
df_comp["delta_1_Healthcare"] = df_comp.Healthcare.diff(1)

In [ ]:
df_comp.head()

In [ ]:
df_comp.delta_1_Healthcare.plot(figsize=(20,5))

In [ ]:
# Checking the normality again with Density Plots
df_comp["delta_1_Healthcare"].plot(kind='kde', figsize=(20, 10))
pyplot.show()

In [ ]:
data_df_res = df_comp[["timestamp", "delta_1_Healthcare"]]

In [ ]:
# train set split
test_size = 12
X = df_comp["timestamp"]
y = df_comp["delta_1_Healthcare"]

x_train_res = X[:-test_size]
y_train_res = y[:-test_size]

x_test_res = X[-test_size:]
y_test_res = y[-test_size:]

df_train_res = data_df_res[:-test_size][1:]
df_test_res = data_df_res[-test_size:][1:]

plt.figure(figsize=(20,5))
plt.title('train and test sets', size=20)
plt.plot(y_train_res, label='Training set')
plt.plot(y_test_res, label='Test set', color='orange')

plt.legend();

In [ ]:
x_train_res_1 = x_train_res.values.reshape(-1, 1)[1:]
y_train_res_1 = y_train_res.values.reshape(-1, 1)[1:]

x_test_res_1 = x_test_res.values.reshape(-1, 1)[1:]
y_test_res_1 = y_test_res.values.reshape(-1, 1)[1:]

In [ ]:
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared, ConstantKernel

k0 = WhiteKernel(noise_level=0.3**2, noise_level_bounds=(0.1**2, 0.5**2))

k1 = ConstantKernel(constant_value=2) * \
  ExpSineSquared(length_scale=1.0, periodicity=40, periodicity_bounds=(35, 45))

kernel_1  = k0 + k1 

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

gp1 = GaussianProcessRegressor(
    kernel=kernel_1, 
    n_restarts_optimizer=5, 
    normalize_y=True,
    alpha=0.0001
)

In [ ]:
gp1.fit(x_train_res_1, y_train_res_1)

In [ ]:
mlf_run2.log_params(gp1.get_params())    # logging params
mlf_run2.log_model(gp1, mlf.ModelFramework.SKLEARN) # log model

In [ ]:
# Generate predictions.
y_pred, y_std = gp1.predict(x_train_res_1, return_std=True)


df_train_res['y_pred'] = y_pred
df_train_res['y_std'] = y_std
df_train_res['y_pred_lwr'] = df_train_res['y_pred'] - 2*df_train_res['y_std']
df_train_res['y_pred_upr'] = df_train_res['y_pred'] + 2*df_train_res['y_std']

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(df_train_res["y_pred"], color='red')
plt.plot(df_train_res["delta_1_Healthcare"] )
plt.show()

In [ ]:
# Generate predictions.
y_pred, y_std = gp1.predict(x_test_res_1, return_std=True)


df_test_res['y_pred'] = y_pred
df_test_res['y_std'] = y_std
df_test_res['y_pred_lwr'] = df_test_res['y_pred'] - 2*df_test_res['y_std']
df_test_res['y_pred_upr'] = df_test_res['y_pred'] + 2*df_test_res['y_std']

In [ ]:
mlf_run.log_predictions(df_test_res[['timestamp', 'delta_1_Healthcare']], list(y_pred))   #logging predictions

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(df_test_res["y_pred"])
plt.plot(df_test_res["delta_1_Healthcare"], color='red')
plt.show()

In [ ]:
df_test_res

In [ ]:
masel = mase_loss(df_test_res["delta_1_Healthcare"], df_test_res["y_pred"].to_list(), df_train_res["delta_1_Healthcare"].to_list())
mlf_run2.log_metrics({'mase_loss': masel})    # logging metrics

import shap
explainer = shap.KernelExplainer(gp1.predict,x_train)
shap_values = explainer.shap_values(x_test)

mlf_run2.log_dataset_stats(
    df_test_res, 
    data_slice=mlf.DataSlice.TEST,
    data_schema=mlf.Schema(
        feature_column_names=['timestamp'],
        prediction_column_name="y_pred",
        actual_column_name="delta_1_Healthcare"
    ),
    model_type=mlf.ModelType.TIMESERIES,
    shap_values=shap_values
)